In [55]:
import pandas as pd 
import fuzzywuzzy
from fuzzywuzzy import fuzz
import numpy as np
import sklearn
from sklearn.utils import shuffle

import recordlinkage
from recordlinkage.datasets import load_febrl4

In [56]:
dfA, dfB = load_febrl4()

In [57]:
len(dfA)

5000

In [58]:
len(dfB)

5000

In [59]:
dfA.columns

Index(['given_name', 'surname', 'street_number', 'address_1', 'address_2',
       'suburb', 'postcode', 'state', 'date_of_birth', 'soc_sec_id'],
      dtype='object')

In [60]:
ones_data = []
zeros_data = []

In [61]:
def create_ones_data(start,end) :
    df1 = dfA.iloc[start:end,:]
    df2 = dfB.iloc[start:end,:]
    
    data = []
    #data1 = []

    for i in range(500) :
        l = []
        #l1=[]
        a = list(df1.iloc[i,:])
        for j in range(500) :
            l = []
            l.extend(a)
            b = list(df2.iloc[j,:])
            if(fuzz.token_set_ratio(a[0],b[0]) > 95.0) :
                l.extend(b)
                data.append(l)
            '''else :
                l1.extend(b)
                data1.append(l)'''
                
    
    data = pd.DataFrame(data)
    #data1 = pd.DataFrame(data1)
    data.columns = ['given_name_A', 'surname_A', 'street_number_A', 'address_1_A', 'address_2_A','suburb_A', 'postcode_A', 'state_A', 'date_of_birth_A', 'soc_sec_id_A','given_name_B', 'surname_B', 'street_number_B', 'address_1_B', 'address_2_B','suburb_B', 'postcode_B', 'state_B', 'date_of_birth_B', 'soc_sec_id_B']
    #s = "created_data_" + str(ctr) + ".csv"
    
    data.to_csv("created_data.csv", index=False)
    d1 = pd.read_csv("created_data.csv")
    d1.fillna('xxxxxxxx')
    
    def dob_conv(dob) :
        k=str(dob)
        
        return k
    
    d1["street_number_A"] = d1["street_number_A"].apply(dob_conv)
    d1["street_number_B"] = d1["street_number_B"].apply(dob_conv)
    
    d1["date_of_birth_A"] = d1["date_of_birth_A"].apply(dob_conv)
    d1["date_of_birth_B"] = d1["date_of_birth_B"].apply(dob_conv)
    
    d1["given_name_A"] = d1["given_name_A"].apply(dob_conv)
    d1["given_name_B"] = d1["given_name_B"].apply(dob_conv)
    
    d1["surname_A"] = d1["surname_A"].apply(dob_conv)
    d1["surname_B"] = d1["surname_B"].apply(dob_conv)
    
    d1["address_1_A"] = d1["address_1_A"].apply(dob_conv)
    d1["address_1_B"] = d1["address_1_B"].apply(dob_conv)
    
    
    d1["address_2_A"] = d1["address_2_A"].apply(dob_conv)
    d1["address_2_B"] = d1["address_2_B"].apply(dob_conv)
    
    d1["suburb_A"] = d1["suburb_A"].apply(dob_conv)
    d1["suburb_B"] = d1["suburb_B"].apply(dob_conv)
    
    d1["state_A"] = d1["state_A"].apply(dob_conv)
    d1["state_B"] = d1["state_B"].apply(dob_conv)
    
    null_value_stats = d1.isnull().sum(axis=0)
    null_value_stats[null_value_stats != 0]
    
    labels = []
    c = d1.columns
    c1 = c[:10]
    c2 = c[10:]
    print(c1,c2)

    c3 = ['given_name_A', 'surname_A', 'date_of_birth_A','soc_sec_id_A'] 
    c4 = ['given_name_B', 'surname_B','date_of_birth_B','soc_sec_id_B']
    
    for i,row in d1.iterrows() :
        l = []
        for j in range(0,len(c3)) :
            s = fuzz.token_set_ratio(row[c3[j]] , row[c4[j]])
            l.append(s)

        cnt = 0
        for p in range(0,len(l)) :
            if(l[p] > 90.0) :
                cnt+=1
        if(cnt >= 3) :
            labels.append(1)
        else :
            labels.append(0)
            
    print(len(d1),len(labels))
    d1["match"] = labels
    print("Number of 1s  " + str(sum(d1["match"] == 1)))
    
    
    
    l1 = []

    for i,r in d1.iterrows() :
        if(r["match"] == 1) :
            ones_data.append(r)
        else :
            zeros_data.append(r)
            
    o = ones_data
    
    return o

In [62]:
for i in range(0,5000,500):
    c = create_ones_data(i,i+500)

Index(['given_name_A', 'surname_A', 'street_number_A', 'address_1_A',
       'address_2_A', 'suburb_A', 'postcode_A', 'state_A', 'date_of_birth_A',
       'soc_sec_id_A'],
      dtype='object') Index(['given_name_B', 'surname_B', 'street_number_B', 'address_1_B',
       'address_2_B', 'suburb_B', 'postcode_B', 'state_B', 'date_of_birth_B',
       'soc_sec_id_B'],
      dtype='object')
992 992
Number of 1s  25
Index(['given_name_A', 'surname_A', 'street_number_A', 'address_1_A',
       'address_2_A', 'suburb_A', 'postcode_A', 'state_A', 'date_of_birth_A',
       'soc_sec_id_A'],
      dtype='object') Index(['given_name_B', 'surname_B', 'street_number_B', 'address_1_B',
       'address_2_B', 'suburb_B', 'postcode_B', 'state_B', 'date_of_birth_B',
       'soc_sec_id_B'],
      dtype='object')
1070 1070
Number of 1s  36
Index(['given_name_A', 'surname_A', 'street_number_A', 'address_1_A',
       'address_2_A', 'suburb_A', 'postcode_A', 'state_A', 'date_of_birth_A',
       'soc_sec_id_A'],


In [63]:
pd.DataFrame(ones_data)
print(len(ones_data))

304


In [64]:
print(len(zeros_data))

10019


In [65]:
od=pd.DataFrame(ones_data)

In [66]:
zd = pd.DataFrame(zeros_data)

In [67]:
zd = shuffle(zd)

In [68]:
zd.to_csv("zeros_file.csv",index = False)

In [69]:
od.to_csv("ones_file.csv",index = False)

In [70]:
od.head()

,given_name_A,surname_A,street_number_A,address_1_A,address_2_A,suburb_A,postcode_A,state_A,date_of_birth_A,soc_sec_id_A,...,surname_B,street_number_B,address_1_B,address_2_B,suburb_B,postcode_B,state_B,date_of_birth_B,soc_sec_id_B,match
80,caitlin,basey,414.0,pelsart street,st francis vlge,daisy hill,2113,nsw,19110711.0,7811311,...,basey,414.0,pelsart street,st francis vlge,daisyh ill,2113,nsw,19110711.0,7811311,1
100,nan,lowry,6.0,hadleigh circuit,rowethorpe,rossmoyne,2770,sa,19350708.0,6616912,...,lowry,80.0,kinchela crescent,rowethorpe,rossmoyne,2770,qld,19350708.0,6616912,1
128,dylan,blacklow,7.0,stenhouse close,flr 5 john flynn medical centre,vermont,2538,nsw,19390222.0,8225337,...,blacklow,100.0,stenhouse close,flr 5 john flynn medical centre,vermony,2538,nsw,19399222.0,8225337,1
195,ella,browne,35.0,tanjil loop,cedarview,berkeley vale,4564,qld,19480726.0,6646608,...,browwne,35.0,tanjil loop,cedavriew,berkeley vale,4564,qld,19480726.0,6646608,1
273,angus,godfrey,1.0,lamington street,ulc 4,pannawonica,5607,vic,19050930.0,8426472,...,godfrey,1.0,lamingtons treet,ulc 4,pannawpnoica,5660,vic,19050930.0,8426472,1


In [71]:
zd.head()

,given_name_A,surname_A,street_number_A,address_1_A,address_2_A,suburb_A,postcode_A,state_A,date_of_birth_A,soc_sec_id_A,...,surname_B,street_number_B,address_1_B,address_2_B,suburb_B,postcode_B,state_B,date_of_birth_B,soc_sec_id_B,match
442,joshua,every,14.0,scoble place,hawkins masonic vlge,gray,4216,nsw,19241220.0,9182595,...,akot,8.0,edmunds place,villal18,dapto,6071,vic,nan,3188786,0
266,emiily,rankine,32.0,blackham street,sapri,campbelltown,5068,vic,19060404.0,3688640,...,mchnery,1.0,hurwooplace,paramatta park,leeton,2228,nsw,19550504.0,1357509,0
369,noah,george,3.0,steinwedel street,glendale stud,gumdale,6020,vic,19270723.0,7682028,...,boyle,11.0,doolandcouhrt,nan,flowerdale,3163,vic,19260331.0,7756654,0
108,ruby,hulley,nan,knaggs crescent,strabane,bayswater north,2283,wa,19201008.0,3808150,...,donalsdon,42.0,nan,glenbrook orchard,appin,2035,tas,19141014.0,3742981,0
93,harrison,boulter,9.0,macgregor street,stonyridge,cecil plains,3070,nsw,19110312.0,5094110,...,snellin g,50.0,sproule circuit,yalkara,emerald,4216,nsw,19410412.0,1348871,0
